In [1]:
# import required module
import os
import cv2
# assign directory
dataset = []
data = dict()
directory = ['/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity/images']
labels = ['COVID','Normal','Pneumonia','Bacterial infection']
cnt = 0
# iterate over files in
# that directory
ids = 0
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)
        
        if os.path.isfile(f):
            im = cv2.imread(f)
            im = cv2.resize(im,(32,32))
            dataset.append([im,ids])
            cnt+=1
        if cnt > 6500:
            break
             
    data.update({labels[ids]: cnt})
    ids+=1


In [ ]:
import matplotlib.pyplot as plt

def show_bar(D):

    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    # # for python 2.x:
    # plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
    # plt.xticks(range(len(D)), D.keys())  # in python 2.x

    plt.show()

# COVID-19 radiography dataset

In [2]:
import numpy as np
print(len(dataset))
x = []
y = []
for i in dataset:
    x.append(i[0])
    y.append(i[1])
x = np.array(x, dtype="float32")
y = np.array(y, dtype="float32")
x.shape

17474


(17474, 32, 32, 3)

In [5]:
len(x)

224

# Imbalanced dataset training

In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
xtr, xts, ytr, yts = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
del x
del y

In [5]:
xtr.shape

(13979, 32, 32, 3)

In [6]:
import torch
xtr = torch.from_numpy(xtr).permute(0,3,1,2)
ytr = ytr.astype(int);
ytr = torch.from_numpy(ytr)

In [7]:
xts = torch.from_numpy(xts)
yts = yts.astype(int);
yts = torch.from_numpy(yts)

In [8]:
import torch
from torch import *
my_dataset = torch.utils.data.TensorDataset(xtr,ytr) # create your datset
my_dataloader = torch.utils.data.DataLoader(my_dataset,batch_size=4,
                                          shuffle=True, num_workers=2)

my_dataset = torch.utils.data.TensorDataset(xts,yts) # create your datset
my_tdataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32)

In [9]:

import torch
import torch.nn as nn
import torch.nn.functional as F

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)

        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)# 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

  

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda')

In [31]:
# Instantiate the model 
model = CNNModel() 
  
# Move the model to the GPU if available 
model.to(device) 

CNNModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=4, bias=True)
)

In [33]:
import torch.optim as optim 

In [34]:
# Define a loss function and optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
batch_size=4
num_epochs=10

losses = [] 
accuracies = [] 
  
# Train the model 
for epoch in range(1,num_epochs): 
    y_pred = []
    for i, (images, labels) in enumerate(my_dataloader): 
# Forward pass 
        images=images.to(device) 
        labels=labels.to(device) 
        outputs = model(images)
        loss = criterion(outputs, labels) 
        y_pred.append(outputs)
        # Backward pass and optimization 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

        _, predicted = torch.max(outputs.data, 1) 
    acc = (predicted == labels).sum().item() / labels.size(0) 
    accuracies.append(acc) 
    losses.append(loss.item()) 
        
    # Evaluate the model on the validation set 	 
    print('Epoch [{}/{}],Loss:{:.4f} ,Accuracy:{:.2f} '.format( 
        epoch+1, num_epochs, loss.item(),acc))

/tmp/ipykernel_42/470274265.py:9: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for epoch in range(1,num_epochs):


Epoch [2.0/10],Loss:0.0633 ,Accuracy:1.00 
Epoch [3.0/10],Loss:0.8623 ,Accuracy:0.67 
Epoch [4.0/10],Loss:0.0101 ,Accuracy:1.00 
Epoch [5.0/10],Loss:0.0707 ,Accuracy:1.00 
Epoch [6.0/10],Loss:0.6798 ,Accuracy:0.67 
Epoch [7.0/10],Loss:0.3456 ,Accuracy:0.67 
Epoch [8.0/10],Loss:0.9706 ,Accuracy:0.67 
Epoch [9.0/10],Loss:0.6270 ,Accuracy:0.67 
Epoch [10.0/10],Loss:0.7984 ,Accuracy:0.67 
Epoch [11.0/10],Loss:0.0214 ,Accuracy:1.00 


# Pneumonia merge

In [ ]:
 
directory = ['/kaggle/input/labeled-chest-xray-images/chest_xray/test/PNEUMONIA',
            '/kaggle/input/labeled-chest-xray-images/chest_xray/train/PNEUMONIA']
cnt = 0
# iterate over files in
# that directory
ids = 0
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)

        if os.path.isfile(f):
            cnt+=1
    x = data['Pneumonia']
    data.update({'Pneumonia': x+cnt})
    ids+=1
print(f'New Pneumonia {cnt}')
print(data)

In [ ]:
D = data

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
# # for python 2.x:
# plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
# plt.xticks(range(len(D)), D.keys())  # in python 2.x

plt.show()

# Covid Merge

In [ ]:
 
directory = ['/kaggle/input/covid-xray-dataset/DATA/DATA/1']
cnt = 0
 
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)

        if os.path.isfile(f):
            cnt+=1
    
   
    x = data['COVID']
    data.update({'COVID': x+cnt})
    ids+=1
print(f'New Covid {cnt}')
print(data)

In [ ]:
show_bar(data)